In [50]:
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMRouterChain, LLMChain
from dotenv import load_dotenv
load_dotenv()
import os, warnings
warnings.filterwarnings('ignore')
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router import MultiPromptChain

In [18]:
api_key = os.environ['OPENAI_API_KEY']

In [4]:
# Student ask a question about Physics
# "how does a magnet work?"
# "explain Feynman diagram"
# INPUT --> ROUTER --> LLM decides which chain to route to --> Chain --> OUTPUT

In [68]:
beginner_template = """
You are a physics teacher who is really focused on beginners and explaining complex concepts in simple to understand terms. 
You assume no prior knowledge.
Here is your question:\n{input}
"""

In [70]:
intermediate_template = """
You are a physics teacher who explains moderately difficult physics topics to intermediate audience members.
You have some prior knowledge.
Here is your question:\n{input}
"""

In [72]:
expert_template = """
You are a physics professor who explains physics topics to advanced audience members. 
You can assume anyone you answer has PhD in physics.
Here is your question:\n{input}
"""

In [74]:
# Route Prompt Information
# [] NAME, DESCRIPTION, TEMPLATE
prompt_info = [
    {
        "name": 'beginner physics',
        "description": 'Answers basic physics questions',
        "template": beginner_template
    },
    {
        "name": 'intermediate physics',
        "description": 'Answers moderate physics questions',
        "template": intermediate_template
    },
    {
        "name": 'advanced physics',
        "description": 'Answers advanced physics questions',
        "template": expert_template
    }
]

In [76]:
llm = ChatOpenAI(api_key=api_key)

In [78]:
destination_chains = {}

for p_info in prompt_info:
    name = p_info['name']
    prompt_template = p_info['template']
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm,prompt=prompt)
    destination_chains[name] = chain

In [80]:
# LLM Chain --> Template

In [82]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm,prompt=default_prompt)

In [84]:
print(MULTI_PROMPT_ROUTER_TEMPLATE)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (must include ```json at the start of the respon

In [86]:
destinations = [f"{p_info['name']}: {p_info['description']}" for p_info in prompt_info]
destinations

['beginner physics: Answers basic physics questions',
 'intermediate physics: Answers moderate physics questions',
 'advanced physics: Answers advanced physics questions']

In [88]:
destinations_str = "\n".join(destinations)
print(destinations_str)

beginner physics: Answers basic physics questions
intermediate physics: Answers moderate physics questions
advanced physics: Answers advanced physics questions


In [90]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
print(router_template)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
beginner physics: Answers basic physics questions
intermediate physics: Answers moderate physics ques

In [92]:
router_prompt = PromptTemplate(template=router_template,
                               input_variables=['input'],
                               output_parser=RouterOutputParser())

In [94]:
router_chain = LLMRouterChain.from_llm(llm=llm,prompt=router_prompt)

In [96]:
chain = MultiPromptChain(router_chain=router_chain,
                         destination_chains=destination_chains,
                         default_chain=default_chain,
                         verbose=True)

In [98]:
chain.run("How do magnets work?")



> Entering new MultiPromptChain chain...
beginner physics: {'input': 'How do magnets work?'}
> Finished chain.


'Magnets work by creating a magnetic field around them. This magnetic field is invisible but it is what allows magnets to attract or repel other objects made of certain materials, like iron or steel. Inside a magnet, there are tiny particles called atoms that have their own magnetic fields. When these atoms are aligned in a specific way, they create a stronger magnetic field. When two magnets are brought close together, their magnetic fields interact with each other, causing them to either be attracted or repelled from each other. This is how magnets work to stick to metal objects or push away from each other.'

In [100]:
chain.run("Explain Feynman Diagrams.")



> Entering new MultiPromptChain chain...
intermediate physics: {'input': 'Explain the concept of Feynman Diagrams.'}
> Finished chain.


'Feynman diagrams are a graphical representation of the interactions between particles in quantum field theory, named after the physicist Richard Feynman. They are used to calculate the probability of different outcomes of particle interactions.\n\nIn a Feynman diagram, particles are represented by lines, with arrows indicating the direction of their movement in time. The interactions between particles are represented by vertices, where lines meet. Each vertex corresponds to a term in the mathematical expression for the interaction.\n\nThe lines in a Feynman diagram can be of different types, representing different types of particles such as fermions (like electrons) or bosons (like photons). The diagram shows how particles move through spacetime, exchanging momentum and energy through interactions.\n\nBy using Feynman diagrams, physicists can calculate the probability amplitudes for different outcomes of particle interactions. This allows them to make predictions about the behavior of

In [102]:
chain.run("Can you describe quantum gravity?")



> Entering new MultiPromptChain chain...
advanced physics: {'input': 'Can you describe quantum gravity?'}
> Finished chain.


'Quantum gravity is a theory that aims to unify the principles of quantum mechanics, which describes the behavior of particles on the smallest scales, with the principles of general relativity, which describes the force of gravity on the largest scales. \n\nIn classical physics, gravity is described by the theory of general relativity, which explains how massive objects like planets and stars curve the fabric of spacetime, creating the force we perceive as gravity. However, when we try to apply the principles of quantum mechanics to gravity, we encounter significant challenges and inconsistencies.\n\nQuantum gravity seeks to reconcile these two theories by describing gravity as a quantum force carried by particles called gravitons. The theory suggests that spacetime is not continuous, but instead made up of tiny, discrete units that interact with each other at the smallest scales.\n\nOne of the leading approaches to quantum gravity is string theory, which posits that the fundamental bu

In [104]:
chain.run("Illustrate Einstein's theory of relativity.")



> Entering new MultiPromptChain chain...
advanced physics: {'input': "Illustrate Einstein's theory of relativity."}
> Finished chain.


"Einstein's theory of relativity is a fundamental pillar of modern physics that revolutionized our understanding of space and time. There are two main components to this theory: the special theory of relativity and the general theory of relativity.\n\nThe special theory of relativity, proposed by Albert Einstein in 1905, is based on two postulates: the laws of physics are the same for all observers in uniform motion, and the speed of light in a vacuum is constant for all observers regardless of their relative motion. From these postulates, Einstein derived the famous equations E=mc^2 and the Lorentz transformations, which describe how time and space are perceived differently by observers in relative motion.\n\nThe general theory of relativity, published by Einstein in 1915, extends the principles of the special theory to include gravity as a curvature of spacetime caused by the presence of mass and energy. In this theory, massive objects like stars and planets warp the fabric of spacet

In [106]:
chain.run("Explain the three laws of motion.")



> Entering new MultiPromptChain chain...
beginner physics: {'input': 'Explain the three laws of motion.'}
> Finished chain.


"Sure! The three laws of motion were formulated by Sir Isaac Newton and they are the foundation of classical mechanics. Let's break them down in simple terms:\n\n1. Newton's First Law of Motion: This law is often called the law of inertia. It states that an object at rest will stay at rest, and an object in motion will stay in motion unless acted upon by an external force. In other words, objects like to keep doing what they're doing unless something changes that.\n\n2. Newton's Second Law of Motion: This law relates the force applied to an object, its mass, and its acceleration. It is summarized by the equation F = ma, where F represents the force applied to an object, m represents the object's mass, and a represents its acceleration. Essentially, the greater the force applied to an object, the greater its acceleration will be, and the more massive an object is, the harder it is to accelerate.\n\n3. Newton's Third Law of Motion: This law states that for every action, there is an equal

In [108]:
chain.run("What are the laws of thermodynamics?")



> Entering new MultiPromptChain chain...
beginner physics: {'input': 'What are the laws of thermodynamics?'}
> Finished chain.


'The laws of thermodynamics are fundamental principles that govern how energy behaves in the universe. There are four laws of thermodynamics, but the first and second laws are the most commonly referred to.\n\nThe first law of thermodynamics, also known as the Law of Conservation of Energy, states that energy cannot be created or destroyed, only transferred or converted from one form to another. This means that the total energy in a closed system remains constant.\n\nThe second law of thermodynamics states that the entropy, or disorder, of an isolated system will always increase over time. This law is often summarized as "heat flows from hot to cold" and explains why processes tend to move towards equilibrium.\n\nThese laws are crucial for understanding how energy moves and changes in systems, from the heat flowing in a cup of coffee to the way engines work. They provide a framework for understanding and predicting the behavior of energy in the universe.'

In [116]:
chain.run("Explain Maxwell's theory on electromagnetic waves.")



> Entering new MultiPromptChain chain...
advanced physics: {'input': "Explain Maxwell's theory on electromagnetic waves."}
> Finished chain.


"Maxwell's theory on electromagnetic waves is a fundamental aspect of classical electromagnetism. In the mid-19th century, James Clerk Maxwell formulated a set of equations known as Maxwell's equations that describe how electric and magnetic fields interact and propagate through space.\n\nOne of the key insights of Maxwell's theory is that changing electric fields can create magnetic fields, and changing magnetic fields can create electric fields. This interplay between electric and magnetic fields allows for the generation and propagation of electromagnetic waves.\n\nMaxwell's equations predict the existence of electromagnetic waves, which are oscillating patterns of electric and magnetic fields that travel through space at the speed of light. These waves can have a wide range of frequencies, from radio waves to gamma rays.\n\nOne of the most famous consequences of Maxwell's theory is the prediction of the existence of light as an electromagnetic wave. This unification of electricity,

In [118]:
chain.run("Describe perturbation theory")



> Entering new MultiPromptChain chain...
intermediate physics: {'input': 'Explain the concept of perturbation theory'}
> Finished chain.


"Perturbation theory is a mathematical method used in physics to solve complex problems by breaking them down into simpler, more manageable parts. It is particularly useful when dealing with systems that cannot be solved exactly, such as those that involve interactions between multiple particles or forces.\n\nThe basic idea behind perturbation theory is to start with a known solution to a simpler problem, and then add small corrections to account for the more complicated interactions at play. These corrections are typically expressed as a series of terms, each representing a higher order correction to the solution.\n\nIn practice, perturbation theory is often used to approximate the behavior of a system by considering how it deviates from a known, idealized case. By systematically adding higher order corrections, physicists can obtain increasingly accurate predictions for the system's behavior.\n\nOverall, perturbation theory is a powerful tool that allows physicists to tackle complex 